# The Agents of FloAI

This notebook shows different kinds of agents that can be built using flo-ai. These agents have their own specialities, and this notebook tries to explain them.

Here are the major types of agents:

1. Agentic Agents (`kind: agentic`): Any agent created in flo-ai by default tries to be an agentic agent. This agent always needs a tool, meaning an agent without a tool if specified as agentic agent whill throw an exception

2. LLM Agents (`kind: llm`): These agents are simply an LLM which can answer any questions asked to it. The agents dont except tools. If tool is passed to an agent of type llm, they are ignored.

3. Tool LLM (`kind: tool`): These agents are just tools or functions that can be executed on the current state. Within the tool will be given the current state as the input, meaning the history of what happened in the flo until now

In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from flo_ai.common.flo_logger import get_logger
from flo_ai.common.flo_langchain_logger import FloLangchainLogger
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

## Setup
Create the Flo session, setup tools

In [2]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')

# Create custom logger and callback handler
custom_logger_new = get_logger("CustomFloSession", "DEBUG")
custom_callback = FloLangchainLogger("CustomFloCallback", "DEBUG")

session = FloSession(
    llm, 
    log_level="DEBUG"
)


session.register_tool(
    name="InternetSearchTool", 
    tool=TavilySearchResults()
)

2024-09-18 11:38:15,760 - SESSION - INFO - New FloSession created with ID: 11acf98c-5fe1-4394-8ec9-51c554a300ca
2024-09-18 11:38:15,762 - SESSION - INFO - Tool 'InternetSearchTool' registered for session 11acf98c-5fe1-4394-8ec9-51c554a300ca


## Agentic Agent (agentic)

Here we are gonna create a simple weather assitant flo agent that can check the whether by an internet searching tool.

As you can see the kind is `agentic`

In [3]:
simple_weather_checking_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: weather-assistant
agent:
    name: Weather Assistant
    kind: agentic
    job: >
      Given the city name you are capable of answering the latest whether this time of the year by searching the internet
    tools:
      - name: InternetSearchTool
"""

In [4]:
flo = Flo.build(session, simple_weather_checking_agent,log_level="DEBUG")

flo.invoke("Whats the whether in california")

2024-09-18 11:38:25,283 - BUILDER - INFO - Building Flo instance from YAML


2024-09-18 11:38:25,681 - COMMON - INFO - Flo instance created for session 11acf98c-5fe1-4394-8ec9-51c554a300ca
2024-09-18 11:38:25,681 - COMMON - INFO - Invoking query for session 11acf98c-5fe1-4394-8ec9-51c554a300ca: Whats the whether in california




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726639688, 'localtime': '2024-09-17 23:08'}, 'current': {'last_updated_epoch': 1726639200, 'last_updated': '2024-09-17 23:00', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 251, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 42, 'cloud': 0, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 11.6, 'heatindex_f': 52.8, 'dewpoint_c': 7.3, 'dewpoint_f': 45.1, 'v

{'messages': [HumanMessage(content='Whats the whether in california')],
 'output': 'The weather in California varies by region, but here are some general details for October 2023:\n\n1. **Current Weather**: As of now, in California City, the temperature is around 17.2°C (63°F) with clear skies. The wind is blowing from the west-southwest at about 6.9 mph, and humidity is at 42%.\n\n2. **Historical Weather**: In Los Angeles, the high temperature reached 92°F on October 5, with humidity peaking at 100% on October 7. The weather has been generally warm throughout the month.\n\n3. **Forecast**: The average temperatures in California during October typically range from highs in the mid-70s to low 80s°F, with cooler nights.\n\nFor more detailed and specific forecasts, you can check local weather services or websites like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).'}

In [6]:
# with logging

from flo_ai.common.flo_langchain_logger import FloLangchainLogger

flo = Flo.build(session, simple_weather_checking_agent)

handler = FloLangchainLogger(log_level="INFO")

config = {
    'callbacks' : [handler]
}

query="Whats the whether in california"

flo.invoke(query=query, config=config)

2024-09-18 11:39:12,102 - BUILDER - INFO - Building Flo instance from YAML
2024-09-18 11:39:12,113 - COMMON - INFO - Flo instance created for session 11acf98c-5fe1-4394-8ec9-51c554a300ca
2024-09-18 11:39:12,114 - COMMON - INFO - Invoking query for session 11acf98c-5fe1-4394-8ec9-51c554a300ca: Whats the whether in california
2024-09-18 11:39:12,121 - FloLangChainLogger - INFO - onChainStart: {'messages': [HumanMessage(content='Whats the whether in california')]}




> Entering new AgentExecutor chain...


2024-09-18 11:39:12,354 - FloLangChainLogger - INFO - onChainStart: {'input': ''}
2024-09-18 11:39:12,367 - FloLangChainLogger - INFO - onChainStart: {'input': ''}
2024-09-18 11:39:12,377 - FloLangChainLogger - INFO - onChainStart: {'input': ''}
2024-09-18 11:39:12,381 - FloLangChainLogger - INFO - onChainStart: {'input': ''}
2024-09-18 11:39:12,384 - FloLangChainLogger - INFO - onChainEnd: []
2024-09-18 11:39:12,387 - FloLangChainLogger - INFO - onChainEnd: {'agent_scratchpad': []}
2024-09-18 11:39:12,389 - FloLangChainLogger - INFO - onChainEnd: {'messages': [HumanMessage(content='Whats the whether in california')], 'intermediate_steps': [], 'agent_scratchpad': []}
2024-09-18 11:39:12,391 - FloLangChainLogger - INFO - onChainStart: {'messages': [HumanMessage(content='Whats the whether in california')], 'intermediate_steps': [], 'agent_scratchpad': []}
2024-09-18 11:39:12,394 - FloLangChainLogger - INFO - onChainEnd: messages=[SystemMessage(content='Given the city name you are capable


Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`




2024-09-18 11:39:17,571 - FloLangChainLogger - INFO - onToolEnd: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726639688, 'localtime': '2024-09-17 23:08'}, 'current': {'last_updated_epoch': 1726639200, 'last_updated': '2024-09-17 23:00', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 251, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 42, 'cloud': 0, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 11.6, 'heatindex_f': 52.8, 'dewpoint_c': 7.3, 'dewpoint_f': 45.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 13.9, 'gust_kp

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726639688, 'localtime': '2024-09-17 23:08'}, 'current': {'last_updated_epoch': 1726639200, 'last_updated': '2024-09-17 23:00', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 251, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 42, 'cloud': 0, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 11.6, 'heatindex_f': 52.8, 'dewpoint_c': 7.3, 'dewpoint_f': 45.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 13.9, 'gust_kph': 22.4}}"}, {'url': 'https://www.timeanddate.com/weather/usa/lo

2024-09-18 11:39:19,778 - FloLangChainLogger - INFO - onLLMEnd: [[ChatGenerationChunk(text='The weather in California varies by region, but here are some general details for October 2023:\n\n1. **Current Weather**: As of now, in California City, the temperature is around 17.2°C (63°F) with clear skies. The wind is blowing from the west-southwest at about 6.9 mph, and humidity is at 42%.\n\n2. **Historical Weather**: In Los Angeles, the high temperature reached 92°F on October 5, with humidity peaking at 100% on October 7. The weather has been generally warm throughout the month.\n\n3. **Forecast**: The average temperatures in California during October typically range from highs in the mid-70s to low 80s°F, with cooler nights.\n\nFor more detailed and specific forecasts, you can check local weather services or websites like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).', generation_info={'fi

The weather in California varies by region, but here are some general details for October 2023:

1. **Current Weather**: As of now, in California City, the temperature is around 17.2°C (63°F) with clear skies. The wind is blowing from the west-southwest at about 6.9 mph, and humidity is at 42%.

2. **Historical Weather**: In Los Angeles, the high temperature reached 92°F on October 5, with humidity peaking at 100% on October 7. The weather has been generally warm throughout the month.

3. **Forecast**: The average temperatures in California during October typically range from highs in the mid-70s to low 80s°F, with cooler nights.

For more detailed and specific forecasts, you can check local weather services or websites like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).

> Finished chain.


{'messages': [HumanMessage(content='Whats the whether in california')],
 'output': 'The weather in California varies by region, but here are some general details for October 2023:\n\n1. **Current Weather**: As of now, in California City, the temperature is around 17.2°C (63°F) with clear skies. The wind is blowing from the west-southwest at about 6.9 mph, and humidity is at 42%.\n\n2. **Historical Weather**: In Los Angeles, the high temperature reached 92°F on October 5, with humidity peaking at 100% on October 7. The weather has been generally warm throughout the month.\n\n3. **Forecast**: The average temperatures in California during October typically range from highs in the mid-70s to low 80s°F, with cooler nights.\n\nFor more detailed and specific forecasts, you can check local weather services or websites like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).'}

## LLM Agent (agentic)

Here we are gonna create a simple llm math assitant flo agent that can check answer any math question

As you can see the kind is `llm`

In [7]:
simple_llm_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: Ask llm anything
    kind: llm
    job: >
      You are a high school maths teacher. Answer any questions the students ask 
"""

In [8]:
flo = Flo.build(session, simple_llm_agent)

flo.invoke("What is pythagorus theorum")
# print(flo.invoke("What is pythagorus theorum"))

2024-09-18 11:39:26,151 - BUILDER - INFO - Building Flo instance from YAML
2024-09-18 11:39:26,163 - COMMON - INFO - Flo instance created for session 11acf98c-5fe1-4394-8ec9-51c554a300ca
2024-09-18 11:39:26,169 - COMMON - INFO - Invoking query for session 11acf98c-5fe1-4394-8ec9-51c554a300ca: What is pythagorus theorum


"Pythagoras' Theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. \n\nThe theorem can be expressed with the formula:\n\n\\[ c^2 = a^2 + b^2 \\]\n\nwhere:\n- \\( c \\) is the length of the hypotenuse,\n- \\( a \\) and \\( b \\) are the lengths of the other two sides.\n\nThis theorem is useful for calculating the length of one side of a right triangle if the lengths of the other two sides are known. Would you like to see an example of how to use it?"

#

In [9]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

class PrintStateTool(BaseTool):
    name = "printStateTool"
    description = "Just print the state"

    def _run(
        self, **kwargs
    ) -> str:
        print(kwargs)
        return "Tool call success"
    
session.register_tool(
    name="printStateTool", 
    tool=PrintStateTool()
)
    
simple_tool_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: tool-to-print-state
    kind: tool
    tools:
        - name: printStateTool
"""

flo = Flo.build(session, simple_tool_agent)

flo.invoke("Print what I am saying")

2024-09-18 11:39:31,032 - SESSION - INFO - Tool 'printStateTool' registered for session 11acf98c-5fe1-4394-8ec9-51c554a300ca
2024-09-18 11:39:31,034 - BUILDER - INFO - Building Flo instance from YAML
2024-09-18 11:39:31,037 - COMMON - INFO - Flo instance created for session 11acf98c-5fe1-4394-8ec9-51c554a300ca
2024-09-18 11:39:31,038 - COMMON - INFO - Invoking query for session 11acf98c-5fe1-4394-8ec9-51c554a300ca: Print what I am saying


{'messages': [HumanMessage(content='Print what I am saying')]}


'Tool call success'